In [1]:
!conda activate HPE

"conda" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [2]:
#!pip install mediapipe opencv-python numpy

In [156]:
import cv2
import mediapipe as mp
import numpy as np
from collections import deque
import time

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
cam_res = [640,480]
cam = 1
kodek = 'h264'
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic

### supportive functions

In [172]:
def calc_angle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    if angle >180.0:
        angle = 360-angle
    return angle

def calc_dist(a,b):
    a = np.array(a)
    b = np.array(b)
    if (a is not None) and (b is not None):
        return np.linalg.norm(a-b)
    return 100000

def get_coords(a, third:bool = False):
    if third:
        return [a.x,a.y,a.z]
    return [a.x,a.y]

def make_suggest(diffl,diffr,diffsh=0,diffear=0):
    suggest = ""
    if diffl>=0.045 or diffr>=0.045:
        suggest = 'Старайтесь не поднимать плечо'
    if diffear>0.04:
        suggest = 'Старайтесь не наклонять голову'
    if diffsh>=0.06:
        suggest = 'Старайтесь не наклоняться' 
    return suggest

### open video

In [4]:
cap = cv2.VideoCapture('videos/forehead.mp4')
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = int(cap.get(cv2.CAP_PROP_FPS))
result_reso = (width//4*3, height//4*3)
writer = cv2.VideoWriter('Video_output.mp4', cv2.VideoWriter_fourcc(*kodek),fps,result_reso)


    
for frame_idx in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
    ret, frame = cap.read()
    if not ret:
      continue
    frame = cv2.resize(frame, result_reso, interpolation=cv2.INTER_LINEAR)
    cv2.imshow('open video', frame)

    writer.write(frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
writer.release()

### use webcam

In [4]:
cap = cv2.VideoCapture(cam)
while cap.isOpened():
    ret, frame = cap.read()

    cv2.imshow('Mediapipe feed', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## NEW

### Universal Functions

In [164]:
def universal_func_web(process_image):
    res = cam_res
    cap = cv2.VideoCapture(cam)
    
    counter = 0
    stage = None
    l_s = deque(maxlen=80)
    r_s = deque(maxlen=80)
    is_left = False
    is_right = False
    suggest = ''

    prev_frame_time = 0
    new_frame_time = 0

    with mp_holistic.Holistic(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            _, image = cap.read()
            image = cv2.flip(image,1)
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            new_frame_time = time.time()
            fps = 1/(new_frame_time-prev_frame_time)
            prev_frame_time = new_frame_time
            print(fps)
            
            results = holistic.process(image)

            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            image, stage, suggest, l_s, r_s, is_right, is_left, counter_condition = process_image(image, res, results, stage, suggest, l_s, r_s, is_left, is_right)
            if counter_condition:
                counter+=1
                
            if stage:
                cv2.putText(image, stage,
                       (10,27),
                       cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),1,cv2.LINE_AA)
            else:
                cv2.putText(image, "Опустите руки",
                       (10,27),
                       cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),1,cv2.LINE_AA)
            
            cv2.putText(image, f"{suggest}",
                    (10,57),
                    cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
            
            cv2.putText(image, f"{counter}",
                (330,63),
                cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2,cv2.LINE_AA)  

            cv2.imshow('MediaPipe Holistic', image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        
    cap.release()
    cv2.destroyAllWindows()

In [40]:
# видео, которые нужно обрабатывать, должны находиться в папке videos
def universal_func_video(process_image, path):
    counter = 0
    stage = None
    l_s = deque(maxlen=80)
    r_s = deque(maxlen=80)
    is_left = False
    is_right = False
    suggest = ''
    
    
    cap = cv2.VideoCapture(f'videos/{path}.mp4')
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    res = (width//4*3, height//4*3)
    writer = cv2.VideoWriter(f'test_{path}.mp4', cv2.VideoWriter_fourcc(*kodek),fps,res)
    with mp_holistic.Holistic(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5,
        static_image_mode=True) as holistic:
        for frame_idx in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
            ret, frame = cap.read()
            if not ret:
                continue
            frame = cv2.resize(frame, res, interpolation=cv2.INTER_LINEAR)
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = holistic.process(image)

            # Draw landmark annotation on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            image, stage, suggest, l_s, r_s, is_right, is_left, counter_condition = process_image(image, res, results, stage, suggest, l_s, r_s, is_left, is_right)
            if counter_condition:
                counter+=1
            if stage:
                cv2.putText(image, stage,
                       (10,27),
                       cv2.FONT_HERSHEY_COMPLEX,0.7,(255,255,255),1,cv2.LINE_AA)
            else:
                cv2.putText(image, "Опустите руки",
                       (10,27),
                       cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),1,cv2.LINE_AA)
            
            cv2.putText(image, f"{suggest}",
                    (10,57),
                    cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
            
            cv2.putText(image, f"{counter}",
                (330,63),
                cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2,cv2.LINE_AA)  

            cv2.imshow('MediaPipe Holistic', image)
            writer.write(image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
    writer.release()    
    cap.release()
    cv2.destroyAllWindows()


### functions

#### to mouth

In [175]:
def process_image_mouth(image, res, results, stage, suggest, l_s, r_s, is_left, is_right):
    l_angle = 0
    r_angle = 0
    l_elbow = [0,0]
    r_elbow = [0,0]
    l_wrist = [0,0]
    r_wrist = [0,0]
    l_shoulder = [0,0]
    r_shoulder = [0,0]
    counter_condition = False
    diffl=0
    diffr=0
    l_dist = 1000
    r_dist = 1000
    
    l_f = [0,0]
    r_f = [0,0]
    mouth = [0,0]
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
        )
        
        landmarks = results.pose_landmarks.landmark
        l_shoulder = get_coords(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER])
        l_elbow = get_coords(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW])
        l_wrist = get_coords(landmarks[mp_pose.PoseLandmark.LEFT_WRIST])

        r_shoulder = get_coords(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER])
        r_elbow = get_coords(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW])
        r_wrist = get_coords(landmarks[mp_pose.PoseLandmark.RIGHT_WRIST])
        l_ear = get_coords(landmarks[mp_pose.PoseLandmark.LEFT_EAR])
        r_ear = get_coords(landmarks[mp_pose.PoseLandmark.RIGHT_EAR])
        nose = get_coords(landmarks[mp_pose.PoseLandmark.NOSE])

        l_s.append(l_shoulder[1])
        r_s.append(r_shoulder[1])
        
        l_m = get_coords(landmarks[mp_holistic.PoseLandmark.MOUTH_LEFT])
        r_m = get_coords(landmarks[mp_holistic.PoseLandmark.MOUTH_RIGHT])

        mouth = [(l_m[0]+r_m[0])/2,(l_m[1]+r_m[1])/2]

        l_angle = calc_angle(l_shoulder,l_elbow,l_wrist)
        r_angle = calc_angle(r_shoulder,r_elbow,r_wrist)
            
            
        
        if results.left_hand_landmarks:
            landmarks = results.left_hand_landmarks.landmark
            l_f = get_coords(landmarks[mp_holistic.HandLandmark.MIDDLE_FINGER_PIP])


        if results.right_hand_landmarks:
            landmarks = results.right_hand_landmarks.landmark
            r_f = get_coords(landmarks[mp_holistic.HandLandmark.MIDDLE_FINGER_PIP])

        l_dist = calc_dist(l_f,mouth)
        r_dist = calc_dist(r_f,mouth)
        l_s_c = abs(l_shoulder[0]-mouth[0])
        r_s_c = abs(r_shoulder[0]-mouth[0])
        l_n_c = abs(l_ear[0]-mouth[0])
        r_n_c = abs(r_ear[0]-mouth[0])
        diffl = max(l_s)-(l_s[-1])
        diffr = max(r_s)-(r_s[-1])
        diffsh = abs(l_s_c-r_s_c)
        diffear = abs(l_n_c-r_n_c)
        center = [(l_shoulder[0]+r_shoulder[0])/2,(l_shoulder[1]+r_shoulder[1])/2]
        
        cv2.putText(image, f"{l_angle:.3f}",
            tuple(np.multiply(l_elbow,res).astype(int)),
            cv2.FONT_HERSHEY_COMPLEX,0.5,(255,255,255),1,cv2.LINE_AA
        )

        cv2.putText(image, f"{r_angle:.3f}",
               tuple(np.multiply(r_elbow,res).astype(int)),
               cv2.FONT_HERSHEY_COMPLEX,0.5,(255,255,255),1,cv2.LINE_AA
            )
        cv2.putText(image, f"{l_dist:.3f}",
                       tuple(np.multiply(l_f[:2],res).astype(int)),
                       cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
        cv2.putText(image, f"{r_dist:.3f}",
                       tuple(np.multiply(r_f[:2],res).astype(int)),
                       cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
              

        cv2.rectangle(image,(0,0),(res[0],73),(245,117,16),-1)
        
        if l_angle > 100 and r_angle > 100 and (stage is None):
                stage = 'Поднесите руку ко рту'

        if (l_angle > 100 and is_left) or (r_angle > 100 and is_right):
            if stage == 'Опустите руку':
                l_s.clear()
                r_s.clear()
            is_left = False
            is_right = False
            stage = 'Поднесите руку ко рту'
            
        suggested_text = make_suggest(diffl,diffr,diffsh,diffear)
        
        if suggested_text!="":
            suggest = suggested_text
                
        if (l_dist<0.035) and (l_angle < 50) and (stage == 'Поднесите руку ко рту'):
            if suggested_text=="":
                suggest = ''
                counter_condition = True
            is_left = True
            stage = f'Опустите руку'
        
        if (r_dist<0.035) and (r_angle < 50) and (stage == 'Поднесите руку ко рту'):
            if suggested_text=="":
                suggest = ''
                counter_condition = True
            is_right = True
            stage = f'Опустите руку'
        
    return (image, stage, suggest, l_s, r_s, is_right, is_left, counter_condition)


In [ ]:
universal_func_web(process_image_mouth)

In [97]:
universal_func_video(process_image_mouth,'mouth_sick')

#### to forehead

In [43]:
def process_image_forehead(image, res, results, stage, suggest, l_s, r_s, is_left, is_right):
    l_angle = 0
    r_angle = 0
    l_elbow = [0,0]
    r_elbow = [0,0]
    l_wrist = [0,0]
    r_wrist = [0,0]
    l_shoulder = [0,0]
    r_shoulder = [0,0]
    counter_condition = False
    diffl=0
    diffr=0
    l_dist = 1000
    r_dist = 1000
    
    l_f = [0,0]
    r_f = [0,0]
    forehead = [0,0]
    
    if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
            )
            
            landmarks = results.pose_landmarks.landmark
            l_shoulder = get_coords(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER])
            l_elbow = get_coords(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW])
            l_wrist = get_coords(landmarks[mp_pose.PoseLandmark.LEFT_WRIST])
            r_shoulder = get_coords(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER])
            r_elbow = get_coords(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW])
            r_wrist = get_coords(landmarks[mp_pose.PoseLandmark.RIGHT_WRIST])
            l_s.append(l_shoulder[1])
            r_s.append(r_shoulder[1])
            
            l_angle = calc_angle(l_shoulder,l_elbow,l_wrist)
            r_angle = calc_angle(r_shoulder,r_elbow,r_wrist)
            
               
            if results.left_hand_landmarks:                   
                    landmarks = results.left_hand_landmarks.landmark
                    l_f = get_coords(landmarks[mp_holistic.HandLandmark.RING_FINGER_PIP])
            if results.right_hand_landmarks:
                    landmarks = results.right_hand_landmarks.landmark
                    r_f = get_coords(landmarks[mp_holistic.HandLandmark.RING_FINGER_PIP])


            if results.face_landmarks:
                landmarks = results.face_landmarks.landmark
                forehead = get_coords(landmarks[151])
                cv2.putText(image, f".",
                   tuple(np.multiply(get_coords(landmarks[151]),res).astype(int)),
                   cv2.FONT_HERSHEY_COMPLEX,0.3,(0,0,255),1,cv2.LINE_AA)

            l_dist = calc_dist(l_f,forehead)
            r_dist = calc_dist(r_f,forehead)
            diffl = max(l_s)-(l_s[-1])
            diffr = max(r_s)-(r_s[-1])
            
    cv2.putText(image, f"{l_angle:.3f}",
            tuple(np.multiply(l_elbow,res).astype(int)),
            cv2.FONT_HERSHEY_COMPLEX,0.5,(255,255,255),1,cv2.LINE_AA
        )
    cv2.putText(image, f"{r_angle:.3f}",
            tuple(np.multiply(r_elbow,res).astype(int)),
            cv2.FONT_HERSHEY_COMPLEX,0.5,(255,255,255),1,cv2.LINE_AA
        )

    cv2.putText(image, f"{l_dist:.3f}",
                    tuple(np.multiply(l_f[:2],res).astype(int)),
                    cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
    cv2.putText(image, f"{r_dist:.3f}",
                    tuple(np.multiply(r_f[:2],res).astype(int)),
                    cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
    

    
    cv2.rectangle(image,(0,0),(res[0],73),(245,117,16),-1)

    if l_angle > 100 and r_angle > 100 and (stage is None):
            stage = 'Поднесите руку ко лбу'

    if (l_angle > 100 and is_left) or (r_angle > 100 and is_right):
        if stage == 'Опустите руку':
            l_s.clear()
            r_s.clear()
        stage = 'Поднесите руку ко лбу'
        is_left = False
        is_right = False

    if diffl>=0.045 or diffr>=0.045:
            suggest = 'Старайтесь не поднимать плечо'
        
    if (l_dist<0.050) and (l_angle < 80) and (stage == 'Поднесите руку ко лбу'):
        if diffl<0.045 and diffr<0.045:
            suggest = ''
            counter_condition = True
        stage = f'Опустите руку'
        is_left = True

    if (r_dist<0.050) and (r_angle < 80) and (stage == 'Поднесите руку ко лбу'):
        if diffl<0.045 and diffr<0.045:
            suggest = ''
            counter_condition = True
        stage = f'Опустите руку'
        is_right = True
        
    return (image, stage, suggest, l_s, r_s, is_right, is_left, counter_condition)


In [33]:
universal_func_video(process_image_forehead,'forehead')

In [44]:
universal_func_web(process_image_forehead)

#### to backhead

In [49]:
def process_image_backhead(image, res, results, stage, suggest, l_s, r_s, is_left, is_right):
    l_angle = 0
    r_angle = 0
    # l_s_angle = 0
    # r_s_angle = 0
    l_elbow = [0,0]
    r_elbow = [0,0]
    l_wrist = [0,0]
    r_wrist = [0,0]
    l_shoulder = [0,0]
    r_shoulder = [0,0]
    counter_condition = False
    diffl=0
    diffr=0
    
    forehead = [0,0]

    if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
            )
            
            landmarks = results.pose_landmarks.landmark
           
            l_wrist = get_coords(landmarks[mp_holistic.PoseLandmark.LEFT_WRIST], True)
            r_wrist = get_coords(landmarks[mp_holistic.PoseLandmark.RIGHT_WRIST], True)
            l_shoulder = get_coords(landmarks[mp_holistic.PoseLandmark.LEFT_SHOULDER], True)
            r_shoulder = get_coords(landmarks[mp_holistic.PoseLandmark.RIGHT_SHOULDER], True)
            l_elbow = get_coords(landmarks[mp_holistic.PoseLandmark.LEFT_ELBOW], True)
            r_elbow = get_coords(landmarks[mp_holistic.PoseLandmark.RIGHT_ELBOW], True)
            l_s.append(l_shoulder[1])
            r_s.append(r_shoulder[1])
            
            # r_s_angle = calc_angle(l_shoulder,r_shoulder,r_elbow)
            # l_s_angle = calc_angle(r_shoulder,l_shoulder,l_elbow)
            l_angle = calc_angle(l_shoulder,l_elbow,l_wrist)
            r_angle = calc_angle(r_shoulder,r_elbow,r_wrist)
            
        
            if results.face_landmarks:
                landmarks = results.face_landmarks.landmark
                forehead = get_coords(landmarks[151])
                cv2.putText(image, f".",
                    tuple(np.multiply(forehead,res).astype(int)),
                    cv2.FONT_HERSHEY_COMPLEX,0.3,(0,0,255),1,cv2.LINE_AA)

            diffl = max(l_s)-(l_s[-1])
            diffr = max(r_s)-(r_s[-1])
            
    cv2.putText(image, f"{l_angle:.3f}",
            tuple(np.multiply(l_elbow[:2],res).astype(int)),
            cv2.FONT_HERSHEY_COMPLEX,0.5,(255,255,255),1,cv2.LINE_AA
        )
    cv2.putText(image, f"{r_angle:.3f}",
            tuple(np.multiply(r_elbow[:2],res).astype(int)),
            cv2.FONT_HERSHEY_COMPLEX,0.5,(255,255,255),1,cv2.LINE_AA
        )
    
    cv2.putText(image, f"{diffl:.3f}",
            tuple(np.multiply(l_shoulder[:2],res).astype(int)),
            cv2.FONT_HERSHEY_COMPLEX,0.5,(255,255,255),1,cv2.LINE_AA
        )
    cv2.putText(image, f"{diffr:.3f}",
            tuple(np.multiply(r_shoulder[:2],res).astype(int)),
            cv2.FONT_HERSHEY_COMPLEX,0.5,(255,255,255),1,cv2.LINE_AA
        )

    
    cv2.rectangle(image,(0,0),(res[0],73),(245,117,16),-1)

    if l_angle > 100 and r_angle > 100 and (stage is None):
            stage = 'Поднесите руку к затылку'

    if (l_angle > 100 and l_shoulder[1]<l_elbow[1] and is_left) or (r_angle > 100 and r_shoulder[1]<r_elbow[1] and is_right):
        if stage == 'Опустите руку':
            l_s.clear()
            r_s.clear()
        stage = 'Поднесите руку к затылку'
        is_left = False
        is_right = False

    if diffl>=0.065 or diffr>=0.065:
            suggest = 'Старайтесь не поднимать плечо'
        

    if (l_wrist[1] > forehead[1] and l_elbow[1] < l_shoulder[1] and l_angle<90) and (stage == 'Поднесите руку к затылку'):
        if diffl<0.065 and diffr<0.065:
            suggest = ''
            counter_condition = True
        is_left = True
        stage = f'Опустите руку'
    if (r_wrist[1] > forehead[1] and r_elbow[1] < r_shoulder[1] and r_angle<90) and (stage == 'Поднесите руку к затылку'):
        if diffl<0.045 and diffr<0.045:
            suggest = ''
            counter_condition = True
        is_right = True
        stage = f'Опустите руку'

    return (image, stage, suggest, l_s, r_s, is_right, is_left, counter_condition)

In [50]:
universal_func_web(process_image_backhead)

In [36]:
universal_func_video(process_image_backhead,'back_head_sick')

## with webcam

In [5]:
cap = cv2.VideoCapture(cam)
with mp_hands.Hands(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        
#         frame = cv2.flip(frame,1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        results = pose.process(image)
        
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if results.multi_hand_landmarks:
            for handlmk in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, handlmk, mp_hands.HAND_CONNECTIONS,
                                         mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                         mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

        cv2.imshow('Mediapipe feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

#### DETERMING JOINTS

In [56]:
cap = cv2.VideoCapture(cam)
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        results = pose.process(image)
        
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

        cv2.imshow('Mediapipe feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

### calculate angle

In [8]:
cap = cv2.VideoCapture(cam)
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
#         frame = cv2.flip(frame,1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        results = pose.process(image)
        
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        try:
            landmarks = results.pose_landmarks.landmark
            shoulder = get_coords(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER])
            l_elbow = get_coords(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW])
            l_wrist = get_coords(landmarks[mp_pose.PoseLandmark.LEFT_WRIST])
            angle = calc_angle(shoulder,l_elbow,l_wrist)
            cv2.putText(image, f"{angle:.3f}",
                       tuple(np.multiply(l_elbow,cam_res).astype(int)),
                       cv2.FONT_HERSHEY_COMPLEX,0.5,(255,255,255),1,cv2.LINE_AA)
        except:
            pass

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

        cv2.imshow('Mediapipe feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()